In [1]:
# Student Name: Muhammad Waqas Aslam
# ID: 775131

# Task 1: Hello World

In [2]:
HelloWorld = """#include <mpi.h>
#include <iostream>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);  // Initialize MPI

    int world_rank;
    int world_size;

    MPI_Comm_rank(MPI_COMM_WORLD, &world_rank);  // Get process rank
    MPI_Comm_size(MPI_COMM_WORLD, &world_size);  // Get number of processes

    std::cout << "Hello from process " << world_rank << " out of " << world_size << " processes." << std::endl;

    MPI_Finalize();  // Finalize MPI
    return 0;
}
"""

In [5]:
cpp_file = open("HelloWorld.cpp", "w")
cpp_file.write(HelloWorld)
cpp_file.close()

!mpic++ HelloWorld.cpp -o HelloWorld
!mpirun -np 2 ./HelloWorld

Hello from process 1 out of 2 processes.
Hello from process 0 out of 2 processes.


# Task 2: Ping-Pong

In [ ]:
PingPong = """#include <mpi.h>
#include <iostream>

int main(int argc, char** argv) {
    // TO-DO
    return 0;
}
"""

In [ ]:
cpp_file = open("PingPong.cpp", "w")
cpp_file.write(PingPong)
cpp_file.close()

!mpic++ PingPong.cpp -o PingPong
!mpirun -np 2 ./PingPong

# Task 3: Collective Communication

# Task 4: Matrix Multiplication

# Task 5: Distributed Search

# Task 6: Extension